# Data Merging Notebook

## Table of Contents

This notebook performs comprehensive data merging operations to combine various datasets for weather-sales analysis.

### 1. Data Import and Setup
- Import required libraries
- Set up file paths and directories
- Load datasets from raw_data and processed_data/Variables folders

### 2. Initial Data Merge
**Step 1**: Merge core datasets
- Merge `umsatzdaten_gekuerzt.csv` (sales data) from raw_data folder
- Combine with variable datasets from processed_data/Variables folder:
  - `Feiertage_variable.csv` (holiday variables)
  - `Niederschlag_variables.csv` (precipitation variables) 
  - `Wetter_variables.csv` (weather variables)
- Perform outer join on `Datum` column
- Output: `merged_umsatzdaten_and_variables.csv`

### 3. Data Quality Assessment
- Check for missing values and data consistency
- Validate date ranges and overlaps
- Identify potential data quality issues

### 4. Final Data Integration
- Additional merging steps (if needed)
- Final data validation and cleaning
- Export final merged dataset

### 5. Summary and Next Steps
- Summary of merged datasets
- Data structure overview
- Recommendations for further analysis

---

## 1. Data Import and Setup

In [4]:
# Import required libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

# Set up file paths
base_path = Path("/home/codespace/team3_goodweather-1/1_DatasetCharacteristics")
raw_data_path = base_path / "raw_data"
processed_data_path = base_path / "processed_data"
variables_path = processed_data_path / "Variables"
output_path = processed_data_path

print(f"Base path: {base_path}")
print(f"Raw data path: {raw_data_path}")
print(f"Variables path: {variables_path}")
print(f"Output path: {output_path}")

# Verify paths exist
for path in [raw_data_path, variables_path, output_path]:
    if path.exists():
        print(f"✓ {path} exists")
    else:
        print(f"✗ {path} does not exist")

Libraries imported successfully!
Base path: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics
Raw data path: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/raw_data
Variables path: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/processed_data/Variables
Output path: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/processed_data
✓ /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/raw_data exists
✓ /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/processed_data/Variables exists
✓ /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/processed_data exists


## 2. Initial Data Merge - Step 1

### Load Individual Datasets
First, we'll load all the datasets that need to be merged:

In [5]:
# Load the main sales data
umsatz_file = raw_data_path / "umsatzdaten_gekuerzt.csv"
print(f"Loading sales data from: {umsatz_file}")
umsatz_df = pd.read_csv(umsatz_file)

# Load variable datasets
feiertage_file = variables_path / "Feiertage_variable.csv"
niederschlag_file = variables_path / "Niederschlag_variables.csv"
wetter_file = variables_path / "Wetter_variables.csv"

print(f"Loading holiday variables from: {feiertage_file}")
feiertage_df = pd.read_csv(feiertage_file)

print(f"Loading precipitation variables from: {niederschlag_file}")
niederschlag_df = pd.read_csv(niederschlag_file)

print(f"Loading weather variables from: {wetter_file}")
wetter_df = pd.read_csv(wetter_file)

# Display basic information about each dataset
datasets = {
    'Sales Data (umsatzdaten_gekuerzt)': umsatz_df,
    'Holiday Variables (Feiertage)': feiertage_df,
    'Precipitation Variables (Niederschlag)': niederschlag_df,
    'Weather Variables (Wetter)': wetter_df
}

print("\n" + "="*60)
print("DATASET OVERVIEW")
print("="*60)

for name, df in datasets.items():
    print(f"\n{name}:")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)}")
    
    # Check for Datum column
    if 'Datum' in df.columns:
        print(f"  Date range: {df['Datum'].min()} to {df['Datum'].max()}")
    else:
        print(f"  ⚠️  No 'Datum' column found!")

Loading sales data from: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/raw_data/umsatzdaten_gekuerzt.csv
Loading holiday variables from: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/processed_data/Variables/Feiertage_variable.csv
Loading precipitation variables from: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/processed_data/Variables/Niederschlag_variables.csv
Loading weather variables from: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/processed_data/Variables/Wetter_variables.csv

DATASET OVERVIEW

Sales Data (umsatzdaten_gekuerzt):
  Shape: (9334, 4)
  Columns: ['id', 'Datum', 'Warengruppe', 'Umsatz']
  Date range: 2013-07-01 to 2018-07-31

Holiday Variables (Feiertage):
  Shape: (2191, 2)
  Columns: ['Datum', 'Feiertage']
  Date range: 2013-01-01 to 2018-12-31

Precipitation Variables (Niederschlag):
  Shape: (2070, 27)
  Columns: ['Datum', 'Niederschlag', 'Month', 'Niederschlag_Lag_1', 'Niederschlag_Lag_2', 'Niedersch

### Perform Outer Join Merges
Now we'll merge all datasets using outer joins on the 'Datum' column:

In [6]:
# Step 1: Start with sales data as the base
print("Step 1: Starting with sales data as base...")
merged_df = umsatz_df.copy()
print(f"Base dataset shape: {merged_df.shape}")

# Step 2: Merge with holiday variables
print("\nStep 2: Merging with holiday variables...")
merged_df = pd.merge(merged_df, feiertage_df, on='Datum', how='outer', suffixes=('', '_feiertage'))
print(f"After holiday merge shape: {merged_df.shape}")

# Step 3: Merge with precipitation variables
print("\nStep 3: Merging with precipitation variables...")
merged_df = pd.merge(merged_df, niederschlag_df, on='Datum', how='outer', suffixes=('', '_niederschlag'))
print(f"After precipitation merge shape: {merged_df.shape}")

# Step 4: Merge with weather variables
print("\nStep 4: Merging with weather variables...")
merged_df = pd.merge(merged_df, wetter_df, on='Datum', how='outer', suffixes=('', '_wetter'))
print(f"Final merged dataset shape: {merged_df.shape}")

print("\n" + "="*60)
print("MERGE RESULTS SUMMARY")
print("="*60)
print(f"Final dataset shape: {merged_df.shape}")
print(f"Date range: {merged_df['Datum'].min()} to {merged_df['Datum'].max()}")
print(f"Total columns: {len(merged_df.columns)}")
print(f"Column names: {list(merged_df.columns)}")

# Check for missing values
print(f"\nMissing values per column:")
missing_values = merged_df.isnull().sum()
for col, missing in missing_values.items():
    if missing > 0:
        percentage = (missing / len(merged_df)) * 100
        print(f"  {col}: {missing} ({percentage:.1f}%)")
    else:
        print(f"  {col}: {missing}")

Step 1: Starting with sales data as base...
Base dataset shape: (9334, 4)

Step 2: Merging with holiday variables...
After holiday merge shape: (9706, 5)

Step 3: Merging with precipitation variables...
After precipitation merge shape: (9706, 31)

Step 4: Merging with weather variables...
Final merged dataset shape: (9706, 151)

MERGE RESULTS SUMMARY
Final dataset shape: (9706, 151)
Date range: 2013-01-01 to 2018-12-31
Total columns: 151
Column names: ['id', 'Datum', 'Warengruppe', 'Umsatz', 'Feiertage', 'Niederschlag', 'Month', 'Niederschlag_Lag_1', 'Niederschlag_Lag_2', 'Niederschlag_Lag_3', 'Niederschlag_Lag_7', 'Niederschlag_Rolling_Mean_3d', 'Niederschlag_Rolling_Sum_3d', 'Niederschlag_Rolling_Mean_7d', 'Niederschlag_Rolling_Sum_7d', 'Niederschlag_Rolling_Mean_14d', 'Niederschlag_Rolling_Sum_14d', 'Has_Rain', 'Heavy_Rain_Day', 'Year', 'DayOfWeek', 'IsWeekend', 'Rain_Category_Extreme_Rain', 'Rain_Category_Heavy_Rain', 'Rain_Category_Light_Rain', 'Rain_Category_Moderate_Rain', 'Rain

### Save Merged Dataset
Save the merged dataset to a new CSV file:

In [7]:
# Save the merged dataset
output_filename = "merged_umsatzdaten_and_variables.csv"
output_filepath = output_path / output_filename

print(f"Saving merged dataset to: {output_filepath}")
merged_df.to_csv(output_filepath, index=False)

# Verify the file was created
if output_filepath.exists():
    file_size = output_filepath.stat().st_size / 1024  # Size in KB
    print(f"✓ File successfully created!")
    print(f"  File size: {file_size:.1f} KB")
    print(f"  Saved {len(merged_df)} rows and {len(merged_df.columns)} columns")
else:
    print("✗ Error: File was not created!")

# Display first few rows of the merged dataset
print(f"\nFirst 5 rows of merged dataset:")
print(merged_df.head())

# Display data types
print(f"\nData types:")
print(merged_df.dtypes)

Saving merged dataset to: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/processed_data/merged_umsatzdaten_and_variables.csv
✓ File successfully created!
  File size: 11984.3 KB
  Saved 9706 rows and 151 columns

First 5 rows of merged dataset:
   id       Datum  Warengruppe  Umsatz  Feiertage  Niederschlag  Month  \
0 NaN  2013-01-01          NaN     NaN          1           NaN    NaN   
1 NaN  2013-01-02          NaN     NaN          0           NaN    NaN   
2 NaN  2013-01-03          NaN     NaN          0           NaN    NaN   
3 NaN  2013-01-04          NaN     NaN          0           NaN    NaN   
4 NaN  2013-01-05          NaN     NaN          0           NaN    NaN   

   Niederschlag_Lag_1  Niederschlag_Lag_2  Niederschlag_Lag_3  ...  \
0                 NaN                 NaN                 NaN  ...   
1                 NaN                 NaN                 NaN  ...   
2                 NaN                 NaN                 NaN  ...   
3               

## 3. Summary of Step 1

### ✅ Completed Tasks:
1. **Data Loading**: Successfully loaded all required datasets:
   - `umsatzdaten_gekuerzt.csv` (9,334 rows, 4 columns)
   - `Feiertage_variable.csv` (2,191 rows, 2 columns)  
   - `Niederschlag_variables.csv` (2,070 rows, 27 columns)
   - `Wetter_variables.csv` (2,056 rows, 121 columns)

2. **Data Merging**: Performed outer joins on 'Datum' column
   - Combined all datasets into one comprehensive dataset
   - Preserved all data from all sources (outer join strategy)

3. **Output Generation**: Created `merged_umsatzdaten_and_variables.csv`
   - File successfully saved to processed_data folder
   - Contains merged sales, holiday, precipitation, and weather data

### 📊 Final Dataset Characteristics:
- **Total columns**: ~150+ variables (original + all engineered features)
- **Date coverage**: Complete date range from all source datasets
- **Missing values**: Present where datasets don't overlap (expected with outer join)

### 🔄 Next Steps:
The table of contents above outlines the remaining steps:
- Data Quality Assessment (Step 3)
- Additional data integration if needed (Step 4) 
- Final summary and analysis preparation (Step 5)

---

*Step 1 Complete: Core datasets successfully merged!*

Loading test.csv...
Test data shape: (1830, 3)
Test data columns: ['id', 'Datum', 'Warengruppe']

First 5 rows of test data:
        id       Datum  Warengruppe
0  1808011  2018-08-01            1
1  1808021  2018-08-02            1
2  1808031  2018-08-03            1
3  1808041  2018-08-04            1
4  1808051  2018-08-05            1

Loading umsatzdaten_gekuerzt.csv...
Umsatz data shape: (9334, 4)
Umsatz data columns: ['id', 'Datum', 'Warengruppe', 'Umsatz']

First 5 rows of umsatz data:
        id       Datum  Warengruppe      Umsatz
0  1307011  2013-07-01            1  148.828353
1  1307021  2013-07-02            1  159.793757
2  1307031  2013-07-03            1  111.885594
3  1307041  2013-07-04            1  168.864941
4  1307051  2013-07-05            1  171.280754

Merging datasets...
Merged data shape: (0, 4)
Merged data columns: ['id', 'Datum', 'Warengruppe', 'Umsatz']

First 5 rows of merged data:
Empty DataFrame
Columns: [id, Datum, Warengruppe, Umsatz]
Index: []

Merge

In [9]:
# Filter out rows without Umsatz values
print("Filtering data to keep only rows with Umsatz values...")
print(f"Original dataset shape: {merged_df.shape}")

# Check how many rows have missing Umsatz values
missing_umsatz = merged_df['Umsatz'].isnull().sum()
total_rows = len(merged_df)
print(f"Rows with missing Umsatz: {missing_umsatz} ({(missing_umsatz/total_rows)*100:.1f}%)")

# Filter to keep only rows with non-null Umsatz values
filtered_df = merged_df.dropna(subset=['Umsatz']).copy()
print(f"Filtered dataset shape: {filtered_df.shape}")

# Show the difference
removed_rows = len(merged_df) - len(filtered_df)
print(f"Removed {removed_rows} rows without Umsatz values")

# Update the output filename and save the filtered dataset
filtered_output_filename = "merged_umsatzdaten_and_variables.csv"
filtered_output_filepath = output_path / filtered_output_filename

print(f"\nSaving filtered dataset to: {filtered_output_filepath}")
filtered_df.to_csv(filtered_output_filepath, index=False)

# Verify the filtered file was created
if filtered_output_filepath.exists():
    file_size = filtered_output_filepath.stat().st_size / 1024  # Size in KB
    print(f"✓ Filtered file successfully created!")
    print(f"  File size: {file_size:.1f} KB")
    print(f"  Saved {len(filtered_df)} rows and {len(filtered_df.columns)} columns")
else:
    print("✗ Error: Filtered file was not created!")

# Display summary of the filtered dataset
print(f"\nFiltered Dataset Summary:")
print(f"Date range: {filtered_df['Datum'].min()} to {filtered_df['Datum'].max()}")
print(f"Umsatz range: {filtered_df['Umsatz'].min():.2f} to {filtered_df['Umsatz'].max():.2f}")
print(f"Average Umsatz: {filtered_df['Umsatz'].mean():.2f}")

# Update the main merged_df variable to the filtered version
merged_df = filtered_df.copy()
print(f"\n✅ Updated merged_df to filtered version with {len(merged_df)} rows")

Filtering data to keep only rows with Umsatz values...
Original dataset shape: (9334, 151)
Rows with missing Umsatz: 0 (0.0%)
Filtered dataset shape: (9334, 151)
Removed 0 rows without Umsatz values

Saving filtered dataset to: /home/codespace/team3_goodweather-1/1_DatasetCharacteristics/processed_data/merged_umsatzdaten_and_variables.csv
✓ Filtered file successfully created!
  File size: 11666.3 KB
  Saved 9334 rows and 151 columns

Filtered Dataset Summary:
Date range: 2013-07-01 to 2018-07-31
Umsatz range: 7.05 to 1879.46
Average Umsatz: 206.75

✅ Updated merged_df to filtered version with 9334 rows
✓ Filtered file successfully created!
  File size: 11666.3 KB
  Saved 9334 rows and 151 columns

Filtered Dataset Summary:
Date range: 2013-07-01 to 2018-07-31
Umsatz range: 7.05 to 1879.46
Average Umsatz: 206.75

✅ Updated merged_df to filtered version with 9334 rows
